In [8]:
import pandas as pd
import time
import requests

base_url = 'http://116.202.111.229:8000'
api_key = 'api-key'

headers = {
    'x-api-key': 'lnzYdcdRr8RiBniVUAXPZlidQAnhDggd'
}
response = requests.get(f"{base_url}/evaluate/hint", headers=headers)

In [9]:
response.json()['hint']

'Lipinski Productions is a Pittsburgh-based company dedicated to providing the highest level of media production services in video editing, media conversion, graphic design, web design, and more....'

  Using cached sentence_transformers-2.7.0-py3-none-any.whl (171 kB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.27.4
    Uninstalling transformers-4.27.4:
      Successfully uninstalled transformers-4.27.4


In [10]:
!pip install sentence-transformers

In [11]:
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer, util
from sklearn.preprocessing import OneHotEncoder
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.preprocessing import OneHotEncoder
import numpy as np

In [12]:
def create_dict(df):
    return dict(zip(df['naics_label'], df['description']))


def eliminate_multimple_spaces_and_newlines(dict):
    for key in dict.keys():
        dict[key] = ' '.join(dict[key].split())
        # eliminate \n
        dict[key] = dict[key].replace('\n', ' ')
    return dict

In [13]:
def prepare_game():
# Initialize tokenizer and model
    model_id = "bert-base-uncased"
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model5 = AutoModelForSequenceClassification.from_pretrained("checkpoint-6500")
    model4 = AutoModelForSequenceClassification.from_pretrained("round4-checkpoint-6500")
    naics = pd.read_excel('Naics3 (label) taxonomy.xlsx')
    naics_dict = create_dict(naics)
    naics_dict = eliminate_multimple_spaces_and_newlines(naics_dict)
    # Load dataset and prepare labels
    full_dataset = pd.read_csv("round5.csv")
    train_dataset, test_dataset = train_test_split(full_dataset, test_size=0.2, random_state=42)
    labels = np.array(train_dataset['predicted']).reshape(-1, 1)  # Ensure labels are correctly shaped

    # Fit the OneHotEncoder once
    encoder = OneHotEncoder(sparse=False)
    encoder.fit(labels)

    model5.eval()
    model4.eval()
    model = SentenceTransformer('all-mpnet-base-v2')
    return model5, model4, tokenizer, encoder, naics_dict, model

In [14]:
model5, model4, tokenizer, encoder, naics_dict, model = prepare_game()
print("Ready to start the game")
abstain = False

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\mihae\anaconda3\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mihae\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Ready to start the game


In [35]:
hint = ""
for i in range(5):
    response = requests.get(f"{base_url}/evaluate/hint", headers=headers)
    time.sleep(2)
    print(i, response.json())
    if hint == "":
        hint = response.json()['hint']
    else:
        hint = hint + ";" + response.json()['hint']
    # Tokenize the input text
    inputs = tokenizer(hint, return_tensors="pt", padding=True, truncation=True, max_length=512)
    # Move inputs to the same device as the model
    inputs = {key: value for key, value in inputs.items()}
    # Perform inference
    with torch.no_grad():
        outputs5 = model5(**inputs)
        logits5 = outputs5.logits

        outputs4 = model4(**inputs)
        logits4 = outputs4.logits
    # Softmax to get probabilities
    probabilities5 = torch.nn.functional.softmax(logits5, dim=-1).cpu().numpy()
    probabilities4 = torch.nn.functional.softmax(logits4, dim=-1).cpu().numpy()
    predicted_index5 = np.argmax(probabilities5, axis=1)
    predicted_one_hot5 = np.zeros(probabilities5.shape)
    predicted_one_hot5[np.arange(len(probabilities5)), predicted_index5] = 1
    predicted_labels5 = encoder.inverse_transform(predicted_one_hot5)
    predicted_labels5 = predicted_labels5[0][0]

    predicted_index4 = np.argmax(probabilities4, axis=1)
    predicted_one_hot4 = np.zeros(probabilities4.shape)
    predicted_one_hot4[np.arange(len(probabilities4)), predicted_index4] = 1
    predicted_labels4 = encoder.inverse_transform(predicted_one_hot4)
    predicted_labels4 = predicted_labels4[0][0]

    # make cosine similarity between the hint and description of the predicted_labels and choose the one with the highest similarity
    query_embedding = model.encode(hint)
    passage_embedding = model.encode(naics_dict[predicted_labels5])
    cosine_scores = util.pytorch_cos_sim(query_embedding, passage_embedding)

    passage_embedding2 = model.encode(naics_dict[predicted_labels4])
    cosine_scores2 = util.pytorch_cos_sim(query_embedding, passage_embedding2)

    if cosine_scores > cosine_scores2:
        predicted_labels = predicted_labels5
    else:
        predicted_labels = predicted_labels4
        cosine_scores = cosine_scores2
    
    print(cosine_scores.item())
    if i + 1 == 5 and cosine_scores.item() < 0.4 and abstain == False:
        abstain = True
        data = {
            'answer': 'abstain'
        }
        response = requests.post(f"{base_url}/evaluate/answer", json=data, headers=headers)
        print(response.status_code, response.json())
    elif i + 1 == 4 and cosine_scores.item() < 0.35 and abstain == False:
        abstain = True
        data = {
            'answer': 'abstain'
        }
        response = requests.post(f"{base_url}/evaluate/answer", json=data, headers=headers)
        print(response.status_code, response.json())
    elif i + 1 == 3 and cosine_scores.item() < 0.3 and abstain == False:
        abstain = True
        data = {
            'answer': 'abstain'
        }
        response = requests.post(f"{base_url}/evaluate/answer", json=data, headers=headers)
        print(response.status_code, response.json())
    elif i + 1 == 2 and cosine_scores.item() < 0.25 and abstain == False:
        abstain = True
        data = {
            'answer': 'abstain'
        }
        response = requests.post(f"{base_url}/evaluate/answer", json=data, headers=headers)
        print(response.status_code, response.json())
    else:
        data = {
            'answer': predicted_labels
        }
        response = requests.post(f"{base_url}/evaluate/answer", json=data, headers=headers)
        print(response.status_code, response.json())
        time.sleep(2)

0 {'company_id': 5, 'level': 5, 'hint': 'Social Services & NGOs'}
0.5497778058052063
200 {'result': 'correct', 'score': 100.0, 'answer': '624 Social Assistance'}
1 {'company_id': 5, 'level': 1, 'hint': 'Heart to Heart'}
0.4787704050540924
200 {'result': 'correct', 'score': 500.0, 'answer': '624 Social Assistance'}
2 {'company_id': 5, 'level': 2, 'hint': 'Global Health Information Provider | Humanitarian Services | Disaster Management Services | Direct Patient Care | Medication Distribution | Hygiene Supplies Distribution | Healthcare Access Improvement | Domestic Health Services'}
0.4971301853656769
200 {'result': 'wrong', 'score': -100.0, 'answer': '624 Social Assistance'}
3 {'company_id': 5, 'level': 3, 'hint': 'As a healthcare nonprofit organization, Heart to Heart International invests in Kansas City to create a healthier world.'}
0.31104329228401184
200 {'result': 'correct', 'score': 300.0, 'answer': '624 Social Assistance'}
4 {'company_id': 5, 'level': 4, 'hint': 'Heart to Heart 